### Analysis of algorithm performance across datasets and metrics for NeurIPS submission

For these tables we use the meta-dataset as for the RecZilla pipeline.

In [28]:
import pandas as pd

In [29]:
df = pd.read_pickle("./meta_datasets/metadata-v1.1.pkl")

In [30]:
df.head()

,alg_family,dataset_name,split_name,original_split_path,hyperparameters_source,time_on_val,time_on_test,time_on_train,test_metric_ARHR_ALL_HITS_cut_1,test_metric_ARHR_ALL_HITS_cut_10,...,param_similarity_from_distance_mode,param_solver,param_symmetric,param_topK,param_total_anneal_steps,param_tversky_alpha,param_tversky_beta,param_use_bias,param_user_reg,alg_param_name
2,UserKNNCF,AmazonElectronicsReader,DataSplitter_leave_k_out_last,gs://reczilla-results/dataset-splits/splits-v5...,default,4583.751314,4548.318467,17298.476997,0.000034,0.000443,...,lin,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,UserKNNCF:euclidean_default
34,TopPop,Jester2Reader,DataSplitter_leave_k_out_last,gs://reczilla-results/dataset-splits/splits-v5...,default,192.594337,193.140875,0.047207,0.053858,0.147120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TopPop:default
137,IALSRecommender,YahooMoviesReader,DataSplitter_leave_k_out_last,gs://reczilla-results/dataset-splits/splits-v5...,default,26.554102,26.568692,606.267164,0.011777,0.031597,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IALSRecommender:default
138,IALSRecommender,YahooMoviesReader,DataSplitter_leave_k_out_last,gs://reczilla-results/dataset-splits/splits-v5...,random_0,26.585350,26.700649,6660.612675,0.000393,0.003225,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IALSRecommender:random_0
139,IALSRecommender,YahooMoviesReader,DataSplitter_leave_k_out_last,gs://reczilla-results/dataset-splits/splits-v5...,random_1,26.952350,26.308905,618.663741,0.009814,0.030576,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IALSRecommender:random_1


### Calculate best metric for each dataset split, over all algs + hyperparam sets.

In [60]:
# define a subset of metrics to focus on

cuts = [1, 2, 5, 10, 50]
metrics = ["NDCG", "PRECISION", "HIT_RATE", "MAP", "F1"]
metric_list = []
metric_name_list = []
for cut in cuts:
    for metric in metrics:
        metric_list.append(f"test_metric_{metric}_cut_{str(cut)}")
        metric_name_list.append(f"{metric}@{cut}")

print(metric_list)

['test_metric_NDCG_cut_1', 'test_metric_PRECISION_cut_1', 'test_metric_HIT_RATE_cut_1', 'test_metric_MAP_cut_1', 'test_metric_F1_cut_1', 'test_metric_NDCG_cut_2', 'test_metric_PRECISION_cut_2', 'test_metric_HIT_RATE_cut_2', 'test_metric_MAP_cut_2', 'test_metric_F1_cut_2', 'test_metric_NDCG_cut_5', 'test_metric_PRECISION_cut_5', 'test_metric_HIT_RATE_cut_5', 'test_metric_MAP_cut_5', 'test_metric_F1_cut_5', 'test_metric_NDCG_cut_10', 'test_metric_PRECISION_cut_10', 'test_metric_HIT_RATE_cut_10', 'test_metric_MAP_cut_10', 'test_metric_F1_cut_10', 'test_metric_NDCG_cut_50', 'test_metric_PRECISION_cut_50', 'test_metric_HIT_RATE_cut_50', 'test_metric_MAP_cut_50', 'test_metric_F1_cut_50']


In [40]:
dataset_col = "dataset_name"

# get global min and max values for each metric, for each dataset
agg_dict = {
    metric: ["min", "max"]
    for metric in metric_list
}
dataset_metric_bounds = df.groupby(dataset_col).agg(agg_dict).reset_index()
dataset_alg_metric_bounds = df.groupby([dataset_col, "alg_family"]).agg(agg_dict).reset_index()

# move to single index - first for global
new_col_names = []
for v in dataset_metric_bounds.columns.values:
    if v[1] in ['min', 'max']:
        new_col_names.append('overall_' + v[1] + '_' + v[0])
    else:
        new_col_names.append(v[0])
        
dataset_metric_bounds.columns = new_col_names

# move to single index - now for dataset-specific
new_col_names = []
for v in dataset_alg_metric_bounds.columns.values:
    if v[1] in ['min', 'max']:
        new_col_names.append(v[1] + '_' + v[0])
    else:
        new_col_names.append(v[0])

dataset_alg_metric_bounds.columns = new_col_names

# now merge in dataset-specific metric bounds with dataset+alg metric bounds
df_expt = dataset_alg_metric_bounds.merge(dataset_metric_bounds, on=dataset_col, how="inner")

# now calculate normalized metric value for each metric
for metric in metric_list:
    df_expt.loc[:, "normalized_" + metric] = (df_expt["max_" + metric] - df_expt["overall_min_" + metric]) / (df_expt["overall_max_" + metric] - df_expt["overall_min_" + metric])

calculate ranks of all algs based on the normalized metrics

In [41]:
# how many nans are there for each metric?
for c in dataset_alg_metric_bounds.columns:
    print(f"{c}: {sum(dataset_alg_metric_bounds[c].isna())}")


dataset_name: 0
alg_family: 0
min_test_metric_NDCG_cut_1: 5
max_test_metric_NDCG_cut_1: 5
min_test_metric_PRECISION_cut_1: 0
max_test_metric_PRECISION_cut_1: 0
min_test_metric_HIT_RATE_cut_1: 0
max_test_metric_HIT_RATE_cut_1: 0
min_test_metric_MAP_cut_1: 0
max_test_metric_MAP_cut_1: 0
min_test_metric_F1_cut_1: 0
max_test_metric_F1_cut_1: 0
min_test_metric_NDCG_cut_2: 8
max_test_metric_NDCG_cut_2: 8
min_test_metric_PRECISION_cut_2: 0
max_test_metric_PRECISION_cut_2: 0
min_test_metric_HIT_RATE_cut_2: 0
max_test_metric_HIT_RATE_cut_2: 0
min_test_metric_MAP_cut_2: 0
max_test_metric_MAP_cut_2: 0
min_test_metric_F1_cut_2: 0
max_test_metric_F1_cut_2: 0
min_test_metric_NDCG_cut_5: 9
max_test_metric_NDCG_cut_5: 9
min_test_metric_PRECISION_cut_5: 0
max_test_metric_PRECISION_cut_5: 0
min_test_metric_HIT_RATE_cut_5: 0
max_test_metric_HIT_RATE_cut_5: 0
min_test_metric_MAP_cut_5: 0
max_test_metric_MAP_cut_5: 0
min_test_metric_F1_cut_5: 0
max_test_metric_F1_cut_5: 0
min_test_metric_NDCG_cut_10: 9
max

In [42]:
# get the rank of each alg within each dataset, for each metric
for metric in metric_list:
    # rank within dataset
    df_expt.loc[:, "rank_" + metric] = df_expt.groupby([dataset_col])["normalized_" + metric].rank(method="min", ascending=False).values


### Table 1: show rank and normalized metrics for all algs, and a sample of datasets

In [51]:

# include cols for these datasets
display_datasets = [
    "Movielens1MReader",
    "Movielens100KReader",
    "Movielens20MReader"
    "AmazonMoviesTVReader",
    "AmazonLuxuryBeautyReader",
    "AmazonWineReader"
    "BookCrossingReader",
    "YahooMoviesReader",
    "CiaoDVDReader",
    "MovieTweetingsReader",
    "AnimeReader",
    "DatingReader",
    # "LastFMReader",  # many NaNs...
    "NetflixPrizeReader",
]

for metric_name in metric_list:

    display_metric = "normalized_" + metric_name
    display_rank = "rank_" + metric_name

    df_expt.loc[:, "display_text"] = df_expt.apply(lambda x: "{:.0f} ({:.2f})".format(x[display_rank], x[display_metric]), axis=1)

    table_1 = df_expt[df_expt["dataset_name"].isin(display_datasets)].pivot(index="alg_family", columns="dataset_name", values="display_text").reset_index()

    new_col_names = [c if ~c.endswith("Reader") else c[:-len("Reader")] for c in table_1.columns]
    new_col_names[0] = "Alg."
    table_1.columns = new_col_names
    
    # export table 1
    csv_name = f"./tables/table_1_{metric_name}.csv"
    tex_name = f"./tables/table_1_{metric_name}.tex"
    table_1.to_csv(csv_name, index=False)
    with open(tex_name, "w") as f:
        table_1.to_latex(f, index=False)
    print(f"wrote table to csv: {csv_name} and tex: {tex_name}")


wrote table to csv: ./tables/table_1_test_metric_NDCG_cut_1.csv and tex: ./tables/table_1_test_metric_NDCG_cut_1.tex
wrote table to csv: ./tables/table_1_test_metric_PRECISION_cut_1.csv and tex: ./tables/table_1_test_metric_PRECISION_cut_1.tex
wrote table to csv: ./tables/table_1_test_metric_HIT_RATE_cut_1.csv and tex: ./tables/table_1_test_metric_HIT_RATE_cut_1.tex
wrote table to csv: ./tables/table_1_test_metric_MAP_cut_1.csv and tex: ./tables/table_1_test_metric_MAP_cut_1.tex
wrote table to csv: ./tables/table_1_test_metric_F1_cut_1.csv and tex: ./tables/table_1_test_metric_F1_cut_1.tex
wrote table to csv: ./tables/table_1_test_metric_NDCG_cut_2.csv and tex: ./tables/table_1_test_metric_NDCG_cut_2.tex
wrote table to csv: ./tables/table_1_test_metric_PRECISION_cut_2.csv and tex: ./tables/table_1_test_metric_PRECISION_cut_2.tex
wrote table to csv: ./tables/table_1_test_metric_HIT_RATE_cut_2.csv and tex: ./tables/table_1_test_metric_HIT_RATE_cut_2.tex
wrote table to csv: ./tables/table

### Table 2: show max and min rank over all datasets for each metric.

In [71]:
# display_datasets_table_2 = [
#     "AmazonAllBeautyReader",
#     "AmazonAllElectronicsReader",
#     "AmazonAlternativeRockReader",
#     "AmazonAmazonFashionReader",
#     "AmazonAmazonInstantVideoReader",
#     "AmazonAppliancesReader",
#     "AmazonAppsforAndroidReader",
#     "AmazonAppstoreforAndroidReader",
#     "AmazonArtsCraftsSewingReader",
#     "AmazonAutomotiveReader",
#     "AmazonBabyReader",
#     "AmazonBabyProductsReader",
#     "AmazonBeautyReader",
#     "AmazonBluesReader",
#     "AmazonBooksReader",
#     "AmazonBuyaKindleReader",
#     "AmazonCDsVinylReader",
#     "AmazonCellPhonesAccessoriesReader",
#     "AmazonChristianReader",
#     "AmazonClassicalReader",
#     "AmazonClothingShoesJewelryReader",
#     "AmazonCollectiblesFineArtReader",
#     "AmazonComputersReader",
#     "AmazonCountryReader",
#     "AmazonDanceElectronicReader",
#     "AmazonDavisReader",
#     "AmazonDigitalMusicReader",
#     "AmazonElectronicsReader",
#     "AmazonFolkReader",
#     "AmazonGiftCardsReader",
#     # "AmazonGospelReader",
#     "AmazonGroceryGourmetFoodReader",
#     "AmazonHardRockMetalReader",
#     "AmazonHealthPersonalCareReader",
#     # "AmazonHomeImprovementReader",
#     "AmazonHomeKitchenReader",
#     "AmazonIndustrialScientificReader",
#     "AmazonInternationalReader",
#     "AmazonJazzReader",
#     "AmazonKindleStoreReader",
#     "AmazonKitchenDiningReader",
#     "AmazonLatinMusicReader",
#     "AmazonLuxuryBeautyReader",
#     "AmazonMagazineSubscriptionsReader",
#     "AmazonMiscellaneousReader",
#     "AmazonMoviesTVReader",
#     "AmazonMP3PlayersAccessoriesReader",
#     "AmazonMusicalInstrumentsReader",
#     "AmazonNewAgeReader",
#     "AmazonOfficeProductsReader",
#     "AmazonOfficeSchoolSuppliesReader",
#     "AmazonPatioLawnGardenReader",
#     "AmazonPetSuppliesReader",
#     "AmazonPopReader",
#     "AmazonPurchaseCirclesReader",
#     "AmazonRapHipHopReader",
#     "AmazonRBReader",
#     "AmazonRockReader",
#     "AmazonSoftwareReader",
#     "AmazonSportsOutdoorsReader",
#     "AmazonToolsHomeImprovementReader",
#     "AmazonToysGamesReader",
#     "AmazonVideoGamesReader",
#     "AmazonWineReader",
#     "Movielens100KReader",
#     "Movielens10MReader",
#     "Movielens1MReader",
#     "Movielens20MReader",
#     "MovielensHetrec2011Reader"
#     "YahooMoviesReader",
#     "YahooMusicReader"
#     "AnimeReader",
#     "BookCrossingReader",
#     "CiaoDVDReader",
#     "DatingReader",
#     "EpinionsReader",
#     "FilmTrustReader",
#     "FrappeReader",
#     "GoogleLocalReviewsReader",
#     "GowallaReader",
#     "Jester2Reader",
#     "LastFMReader",
#     "MarketBiasAmazonReader",
#     "MarketBiasModClothReader",
#     "MovieTweetingsReader",
#     "NetflixPrizeReader",
#     "RecipesReader",
#     "WikilensReader",
# ]

In [74]:
# first show the number of algs with results for each dataset and each metric. (equal to max rank)
# only keep datasets which have at least 10 ranked algs for each metric
max_rank_table = df_expt.groupby("dataset_name")[["rank_" + m for m in metric_list]].max().reset_index()
drop_datasets = []
for c in max_rank_table.columns[1:]:
    if max_rank_table[c].min() < 10:
        # print(f"WARNING: column {c} has a dataset with only {max_rank_table[c].min()} algs")
        # print(max_rank_table[max_rank_table[c] < 10][["dataset_name", c]])
        # print(f"adding to drop_datasets...")
        drop_datasets.extend(max_rank_table[max_rank_table[c] < 10]["dataset_name"])

drop_datasets = set(drop_datasets)
print(f"datasets to drop: {drop_datasets}")

df_table_2 = df_expt.loc[~df_expt["dataset_name"].isin(drop_datasets), :]

print(f"remaining datasets: {len(df_table_2['dataset_name'].unique())}")


datasets to drop: {'AmazonNewAgeReader', 'AmazonCountryReader', 'AmazonHomeImprovementReader', 'YahooMusicReader', 'AmazonAllBeautyReader', 'AmazonFolkReader', 'GowallaReader', 'AmazonBluesReader', 'AmazonHardRockMetalReader', 'AmazonGospelReader', 'AmazonMP3PlayersAccessoriesReader', 'FrappeReader', 'LastFMReader', 'AmazonRapHipHopReader', 'AmazonAllElectronicsReader', 'AmazonCollectiblesFineArtReader', 'AmazonOfficeSchoolSuppliesReader', 'AmazonRockReader', 'AmazonRBReader', 'AmazonAlternativeRockReader', 'AmazonAppstoreforAndroidReader', 'AmazonLatinMusicReader', 'AmazonInternationalReader', 'AmazonComputersReader', 'AmazonWineReader', 'AmazonPopReader'}
remaining datasets: 59


In [80]:
# get min and max rank for each alg and each metric over all algs.     each alg = col, row = metric
agg_dict = {
    "rank_" + metric: lambda x: f"{min(x)} / {max(x)}" for metric in metric_list
}
table_2_transpose = df_table_2.groupby("alg_family").agg(agg_dict).reset_index()

table_2 = table_2_transpose.transpose()

# now fix the metric names